In [1]:
!pip install transformers
!pip install torch

In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [28]:
tokenizer = AutoTokenizer.from_pretrained("roberta-base", padding="max_length", truncation=True)
factkb = AutoModelForSequenceClassification.from_pretrained("bunsenfeng/FactKB", num_labels = 2)

def calc_fact(row):
  input = [[row['output'], row['linearized_input']]]

  tokens = tokenizer(input, return_tensors="pt", padding="max_length", truncation=True)
  result = torch.softmax(factkb(**tokens).logits, dim = 1)

  return float(result[0][1])

In [5]:
summary = "An elephant has been hit by a stone at a zoo in western france after it was hit by a tree."
article = "The stone got past the elephant's fence and a ditch separating the animal and visitors, the zoo said in a statement.The girl was taken to hospital and died within a few hours, the zoo added.The zoo statement said the enclosure met international standards and said 'this kind of accident is rare, unpredictable and unusual'.Africa Live: More on this and other storiesThe statement went on (in French) to point out two other recent incidents in the US:Phyllis Lee, Scientific Director of the Amboseli Trust for Elephants, says that targeted throwing of stones and branches by elephants is very unusual.'It can happen when elephants are frustrated or bored. In my opinion, it's unlikely the elephant was directly targeting the girl - but exhibiting frustration. You can't predict what animals in captivity will do.'The moments after the girl was struck at Rabat Zoo on Tuesday were filmed by a bystander and uploaded onto YouTube.The video shows the elephant waving its trunk behind a fence and swerves round to show a stone on the ground.Metres away people are gathered around the girl, holding her head and stroking her leg."

In [6]:
calc_fact(summary, article)

0.0039005507715046406

In [22]:
lin = "Items for infection control in delivery service area | (Waste receptacle, 55) (Soap and running water or else alcohol-based hand disinfectant, 67) (Latex gloves, 86) (Sharps container, 93)"
out = "Gloves and sharps container were found in more than 90 percent of facilities."
calc_fact(out, lin)  # 0

0.0046424828469753265

In [23]:
lin = "Trends in vaccination coverage among children age 12-23 months* | Percentage of children with specific vaccines | (2008-09 KDHS, All basic vaccinations, 77) (2014 KDHS, All basic vaccinations, 79) (2008-09 KDHS, BCG, 97) (2014 KDHS, BCG, 96) (2008-09 KDHS, DPT1-HepB-Hib, 96) (2014 KDHS, DPT1-HepB-Hib, 98) (2008-09 KDHS, DPT2-HepB-Hib, 93) (2014 KDHS, DPT2-HepB-Hib, 96) (2008-09 KDHS, DPT3-HepB-Hib, 86) (2014 KDHS, DPT3-HepB-Hib, 90) (2008-09 KDHS, Polio 0, 59) (2014 KDHS, Polio 0, 66) (2008-09 KDHS, Polio 1, 96) (2014 KDHS, Polio 1, 98) (2008-09 KDHS, Polio 2, 94) (2014 KDHS, Polio 2, 96) (2008-09 KDHS, Polio 3, 88) (2014 KDHS, Polio 3, 90) (2008-09 KDHS, Measles, 85) (2014 KDHS, Measles, 87)"
out = "The proportion of children receiving all basic vaccinations (93%) are higher in Ruvuma (38%) and Geita (38%) regions."
calc_fact(out, lin)  # 0

0.10390573740005493

In [25]:
import pandas as pd

In [26]:
df = pd.read_csv('test_stata.csv')
df = df[df['lang'] == 'en']

In [29]:
df['prediction'] = df.apply(calc_fact, axis=1)

In [30]:
import pandas as pd
from scipy.stats import pearsonr

def calculate_pearson_correlation(df, column1, column2):
    # Calculate Pearson correlation coefficient
    correlation_coefficient, _ = pearsonr(df[column1], df[column2])

    print(correlation_coefficient)

# Calculate Pearson correlation coefficient between 'Column1' and 'Column2'
correlation = calculate_pearson_correlation(df, 'attributable', 'prediction')

0.22214220877337562
